# Jupyter Notebook

## Prerequisites

AIMBAT can run in a Jupyter Notebook. However, as AIMBAT uses interactive GUI elements, we must specify
that we want those to open inside the notebook rather than independant windows. For matplotlib this is
done by using the Ipython magic command :

In [1]:
%matplotlib widget

PyQtGraph requires an application to be instantiated:

In [2]:
import pyqtgraph as pg

_ = pg.mkQApp()

qt.core.qobject.connect: QObject::connect(QStyleHints, QStyleHints): unique connections require a pointer to member function of a QObject subclass


## Environment Setup

For this example project we create a temporary directory to store seismogram data, and a temporary file to store the aimbat project.

In [3]:
import tempfile
import os

example_directory = tempfile.mkdtemp(prefix="aimbat_dir_")
example_project = tempfile.mktemp(prefix="aimbat_project_", suffix=".db")
os.environ["AIMBAT_PROJECT"] = example_project

<div class="admonition note">
    <p class="admonition-title">Note</p>
    <p>
        By default, all AIMBAT commands assume the project a file called "aimbat.db" in the current directory. Here we override that behavior by settings
    </p>
</div>

## Create a new Project

In [4]:
from aimbat.lib.project import create_project, delete_project

In [5]:
create_project()

Creating an AIMBAT project creates a new sqlite file to store filepaths and processing data in. This also includes some defaults (which can be set on a per project basis too).

In [6]:
from aimbat.lib.db import engine

In [7]:
from aimbat.lib.defaults import (
    get_default,
    set_default,
    print_defaults_table,
)

In [8]:
from sqlmodel import Session

with Session(engine) as session:
    print_defaults_table(session)

KeyError: 'aimbat'

Let's change the `sampledata_dir` to the temporary location we created earlier:

In [ ]:
with Session(engine) as session:
    set_default(session, "sampledata_dir", example_directory)
    print(
        f"The sampledata will be downloaded to {get_default(session, "sampledata_dir")}."
    )

In [ ]:
from aimbat.lib.utils.sampledata import download_sampledata, delete_sampledata
from glob import glob

with Session(engine) as session:
    download_sampledata(session)
    sampledata_directory = get_default(session, "sampledata_dir")
    sacfiles = glob(sampledata_directory + "/**/*.BHZ", recursive=True)
    print(f"Downloaded {len(sacfiles)} files")

In [ ]:
from aimbat.lib.data import add_files_to_project

add_files_to_project(session, sacfiles, "sac")

In [ ]:
from aimbat.lib.utils import plotseis

plotseis(session, event_id=1)
plotseis(session, 1, use_qt=True)

In [ ]:
with Session(engine) as session:
    delete_sampledata(session)

In [ ]:
delete_project()